In [57]:
import pandas as pd

In [3]:
bike_realtime_df = pd.read_csv('./bike_realtime0416-0519.csv')
mrt_history_df = pd.read_csv('./mrt_history_groupby03-04.csv')
time_table_df = pd.read_csv('./time_table.csv')
youbike_mrt_distance_df = pd.read_csv('./youbike_mrt_distance.csv')
bike_station_df = pd.read_csv('./bike_station.csv')
mrt_station_info_df = pd.read_csv('./mrt_station_info.csv')

In [4]:
bike_realtime_df.drop(columns="create_time",inplace=True)

In [5]:
bike_realtime_df["source_time"] = pd.to_datetime(bike_realtime_df["source_time"]) 
bike_realtime_df["date"] = bike_realtime_df["source_time"].dt.date
bike_realtime_df["hour"] = bike_realtime_df["source_time"].dt.hour
bike_realtime_df["mins"] = bike_realtime_df["source_time"].dt.minute

In [6]:
bike_realtime_df["date"] = pd.to_datetime(bike_realtime_df["date"])
time_table_df["date"] = pd.to_datetime(time_table_df["date"])
mrt_history_df["date"] = pd.to_datetime(mrt_history_df["date"])

In [7]:
aval_bike_mean = pd.DataFrame(bike_realtime_df.groupby(["bike_station_id","date","hour"])["aval_bike"].mean()).reset_index(drop=False)
aval_bike_mean.rename(columns={"aval_bike":"aval_bike_mean"},inplace=True)
aval_space_mean = pd.DataFrame(bike_realtime_df.groupby(["bike_station_id","date","hour"])["aval_space"].mean()).reset_index(drop=False)
aval_space_mean.rename(columns={"aval_space":"aval_space_mean"},inplace=True)


In [8]:
df_bike_hour_grby = aval_bike_mean.merge(aval_space_mean,
                     how="left",
                     left_on=["bike_station_id","date","hour"],
                     right_on=["bike_station_id","date","hour"])

In [9]:
df  = df_bike_hour_grby.merge(bike_station_df.loc[:,["bike_station_id","total_space","lat","lng"]],
                              how="left",
                              on="bike_station_id")

In [10]:
time_table_df.loc[0,"date"]

Timestamp('2023-01-01 00:00:00')

In [11]:
df.loc[0,"date"]

Timestamp('2024-04-16 00:00:00')

In [12]:
time_table_df.loc[0,"date"]

Timestamp('2023-01-01 00:00:00')

In [13]:
df = df.merge(time_table_df.loc[:,["date","day_of_week","day_of_week_name"]],
            on="date",
            how="left")


In [23]:
idx = youbike_mrt_distance_df.groupby("bike_station_id")["distance"].idxmin()
youbike_min_distance_table = youbike_mrt_distance_df.loc[idx,].reset_index(drop=True)

In [30]:
mrt_station_table = youbike_min_distance_table.merge(mrt_station_info_df.loc[:,["mrt_station_id","station_name"]],
                                 on="mrt_station_id",
                                 how="left")
mrt_station_table

,bike_station_id,mrt_station_id,distance,station_name
0,500101001,BR08,0.015941,科技大樓
1,500101002,BR08,0.053192,科技大樓
2,500101003,BR08,0.302305,科技大樓
3,500101004,BR08,0.296290,科技大樓
4,500101005,BR08,0.510676,科技大樓
...,...,...,...,...
1410,500119087,G07,0.645005,公館
1411,500119088,BR08,0.683471,科技大樓
1412,500119089,G07,0.884494,公館
1413,500119090,G07,0.719453,公館


In [33]:
df

,bike_station_id,date,hour,aval_bike_mean,aval_space_mean,total_space,lat,lng,day_of_week,day_of_week_name
0,500101001,2024-04-16,22,7.833333,20.166667,28,25.02605,121.54360,3,Tuesday
1,500101001,2024-04-16,23,23.000000,5.000000,28,25.02605,121.54360,3,Tuesday
2,500101001,2024-04-17,0,20.500000,7.500000,28,25.02605,121.54360,4,Wednesday
3,500101001,2024-04-17,1,16.666667,11.333333,28,25.02605,121.54360,4,Wednesday
4,500101001,2024-04-17,2,18.000000,10.000000,28,25.02605,121.54360,4,Wednesday
...,...,...,...,...,...,...,...,...,...,...
1012257,500119091,2024-05-19,19,11.500000,6.500000,18,25.01816,121.54469,1,Sunday
1012258,500119091,2024-05-19,20,12.500000,5.500000,18,25.01816,121.54469,1,Sunday
1012259,500119091,2024-05-19,21,15.500000,2.500000,18,25.01816,121.54469,1,Sunday
1012260,500119091,2024-05-19,22,15.500000,2.500000,18,25.01816,121.54469,1,Sunday


In [35]:
df = df.merge(mrt_station_table,on="bike_station_id",how="left")
df.head()

,bike_station_id,date,hour,aval_bike_mean,aval_space_mean,total_space,lat,lng,day_of_week,day_of_week_name,mrt_station_id,distance,station_name
0,500101001,2024-04-16,22,7.833333,20.166667,28,25.02605,121.5436,3,Tuesday,BR08,0.015941,科技大樓
1,500101001,2024-04-16,23,23.000000,5.000000,28,25.02605,121.5436,3,Tuesday,BR08,0.015941,科技大樓
2,500101001,2024-04-17,0,20.500000,7.500000,28,25.02605,121.5436,4,Wednesday,BR08,0.015941,科技大樓
3,500101001,2024-04-17,1,16.666667,11.333333,28,25.02605,121.5436,4,Wednesday,BR08,0.015941,科技大樓
4,500101001,2024-04-17,2,18.000000,10.000000,28,25.02605,121.5436,4,Wednesday,BR08,0.015941,科技大樓


In [45]:
mrt_history_df = mrt_history_df.merge(time_table_df.loc[:,["date","day_of_week",]],
                                        on="date",
                                        how="left")

In [58]:
mrt_history_df_group_byweek_enter = pd.DataFrame(mrt_history_df.groupby(["mrt_station","day_of_week","hour"])["enter_num"].mean()).reset_index(drop=False)
mrt_history_df_group_byweek_exit = pd.DataFrame(mrt_history_df.groupby(["mrt_station","day_of_week","hour"])["exit_num"].mean()).reset_index(drop=False)

In [61]:
mrt_history_df_group_byweek = mrt_history_df_group_byweek_enter.merge(
    mrt_history_df_group_byweek_exit,
    left_on = ["mrt_station","day_of_week","hour"],
    right_on = ["mrt_station","day_of_week","hour"],
    how = "left"
)
mrt_history_df_group_byweek

,mrt_station,day_of_week,hour,enter_num,exit_num
0,七張,1,0,34.777778,112.444444
1,七張,1,1,0.666667,9.111111
2,七張,1,5,0.222222,0.222222
3,七張,1,6,144.888889,85.555556
4,七張,1,7,328.333333,160.222222
...,...,...,...,...,...
17341,龍山寺,7,19,1731.888889,1695.111111
17342,龍山寺,7,20,1414.444444,1505.888889
17343,龍山寺,7,21,1342.555556,1470.444444
17344,龍山寺,7,22,893.222222,1232.111111


In [63]:
df = df.merge(
    mrt_history_df_group_byweek.loc[:,["day_of_week","hour","mrt_station","enter_num",	"exit_num"]],
    left_on=["day_of_week","hour",	"station_name"],
    right_on=["day_of_week","hour", "mrt_station"],
    how="left"
)

In [66]:
df

,bike_station_id,date,hour,aval_bike_mean,aval_space_mean,total_space,lat,lng,day_of_week,day_of_week_name,mrt_station_id,distance,station_name,mrt_station,enter_num,exit_num
0,500101001,2024-04-16,22,7.833333,20.166667,28,25.02605,121.54360,3,Tuesday,BR08,0.015941,科技大樓,科技大樓,624.444444,402.222222
1,500101001,2024-04-16,23,23.000000,5.000000,28,25.02605,121.54360,3,Tuesday,BR08,0.015941,科技大樓,科技大樓,198.555556,136.555556
2,500101001,2024-04-17,0,20.500000,7.500000,28,25.02605,121.54360,4,Wednesday,BR08,0.015941,科技大樓,科技大樓,82.500000,49.750000
3,500101001,2024-04-17,1,16.666667,11.333333,28,25.02605,121.54360,4,Wednesday,BR08,0.015941,科技大樓,科技大樓,0.500000,0.000000
4,500101001,2024-04-17,2,18.000000,10.000000,28,25.02605,121.54360,4,Wednesday,BR08,0.015941,科技大樓,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012257,500119091,2024-05-19,19,11.500000,6.500000,18,25.01816,121.54469,1,Sunday,BR08,0.890423,科技大樓,科技大樓,624.222222,509.444444
1012258,500119091,2024-05-19,20,12.500000,5.500000,18,25.01816,121.54469,1,Sunday,BR08,0.890423,科技大樓,科技大樓,550.111111,462.666667
1012259,500119091,2024-05-19,21,15.500000,2.500000,18,25.01816,121.54469,1,Sunday,BR08,0.890423,科技大樓,科技大樓,429.666667,473.333333
1012260,500119091,2024-05-19,22,15.500000,2.500000,18,25.01816,121.54469,1,Sunday,BR08,0.890423,科技大樓,科技大樓,273.222222,351.000000


In [68]:
df["bike_rate"] = df["aval_bike_mean"]/df["total_space"]

In [70]:
df["enter_num"].fillna(0,inplace=True)
df["exit_num"].fillna(0,inplace=True)

In [71]:
df.to_csv("test.csv",index=False,encoding="utf-8-sig")

In [ ]:
df.loc